# LLM Zoomcamp Homework 02

## Q1. Embedding the query

Embed the query: <i>'I just discovered the course. Can I join now?'</i>. Use the `jinaai/jina-embeddings-v2-small-en` model.

You should get a numpy array of size 512.

What's the minimal value in this array?

In [1]:
# !pip install fastembed

from fastembed import TextEmbedding
model_handle = 'jinaai/jina-embeddings-v2-small-en'
embedding_model = TextEmbedding(model_handle)

In [2]:
query = 'I just discovered the course. Can I join now?'

emb_query = list(embedding_model.embed(query))[0]
print(len(emb_query))
round(min(emb_query), 2)

512


-0.12

## Q2. Cosine similarity with another vector



In [3]:
import numpy as np

print(np.linalg.norm(emb_query))
round(emb_query.dot(emb_query), 5)

1.0


1.0

In [4]:
doc = 'Can I still join the course after the start date?'
emb_doc = list(embedding_model.embed(doc))[0]

def cos_sim(v1, v2):
    return (v1.dot(v2))/(np.linalg.norm(v1)*np.linalg.norm(v2))

round(cos_sim(emb_query, emb_doc), 2)


0.9

## Q3. Ranking by cosine

Compute the embeddings for the text field, and compute the cosine between the query vector and all the documents.

What's the document index with the highest similarity? (Indexing starts from 0)

In [5]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [6]:
emb_docs = embedding_model.embed([doc['text'] for doc in documents])
emb_matrix = np.array(list(emb_docs))
print([np.linalg.norm(v) for v in emb_matrix])
cos_sims = emb_matrix @ emb_query
dists = {i:cos_sims[i] for i in range(len(cos_sims))}
dists_sorted = dict(sorted(dists.items(), key=lambda x: x[1]))
dists_sorted

[1.0, 1.0, 1.0, 1.0, 1.0]


{3: 0.7133078832064157,
 4: 0.7304499196411822,
 0: 0.7629684518721929,
 2: 0.8085397445747489,
 1: 0.8182378156620137}

## Q4. Ranking by cosine, version two

Now let's calculate a new field, which is a concatenation of `question` and `text`:

`full_text = doc['question'] + ' ' + doc['text']`

Embed this field and compute the cosine between it and the query vector. What's the highest scoring document?

In [7]:
documents_full = [doc['question'] + ' ' + doc['text'] for doc in documents]
documents_full

["Course - Can I still join the course after the start date? Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'Course - Can I follow the course after it finishes? Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
 "Course - When will the course start? The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin t

In [8]:
emb_docs = embedding_model.embed(documents_full)
emb_matrix = np.array(list(emb_docs))
print([np.linalg.norm(v) for v in emb_matrix])
cos_sims = emb_matrix @ emb_query
dists = {i:cos_sims[i] for i in range(len(cos_sims))}
dists_sorted = dict(sorted(dists.items(), key=lambda x: x[1]))
dists_sorted

[0.9999999999999999, 0.9999999999999999, 1.0, 0.9999999999999999, 0.9999999999999999]


{3: 0.7755157657626952,
 4: 0.8086007917931164,
 2: 0.8408287224005013,
 1: 0.8436594005975434,
 0: 0.8514543194432261}

## Q5. Selecting the embedding model
Now let's select a smaller embedding model. What's the smallest dimensionality for models in fastembed?

In [9]:
min_dim = min([model['dim'] for model in TextEmbedding.list_supported_models()])
min_dim


384

In [10]:
import json

for model in TextEmbedding.list_supported_models():
    if model["dim"] == min_dim:
        print(json.dumps(model['model'], indent=2))


"BAAI/bge-small-en"
"BAAI/bge-small-en-v1.5"
"snowflake/snowflake-arctic-embed-xs"
"snowflake/snowflake-arctic-embed-s"
"sentence-transformers/all-MiniLM-L6-v2"
"sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"


## Q6. Indexing with qdrant

For the last question, we will use more documents.

We will select only FAQ records from our ml zoomcamp

In [11]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

Add them to qdrant using the model form Q5.

When adding the data, use both question and answer fields:

In [12]:
# !sudo docker run --rm -p 6333:6333 -p 6334:6334    -v "$(pwd)/qdrant_storage:/qdrant/storage:z"    qdrant/qdrant

In [13]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333") 
collection_name = "collection_1"

client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=min_dim,  
        distance=models.Distance.COSINE  
    )
)

points = []
id = 0

model_small = "BAAI/bge-small-en"

for doc in documents:
   point = models.PointStruct(
      id=id,
      vector=models.Document(text=doc['question'] + ' ' + doc['text'], model=model_small), 
      payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": doc['course'],
            
      } #save all needed metadata fields
   )
   points.append(point)

   id += 1


client.upsert(
    collection_name=collection_name,
    points=points
)


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

After the data is inserted, use the question from Q1 for querying the collection.

What's the highest score in the results? (The score for the first returned record)

In [14]:
results = client.query_points(
    collection_name=collection_name,
    query=models.Document( 
        text=query,
        model=model_small 
    ),
    limit=1,
    with_payload=True 
)

print(f'Max. score: {round(results.points[0].score, 2)}')

Max. score: 0.87
